<a href="https://colab.research.google.com/github/AithaDhanush/majorproject/blob/main/RBF_back_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.cluster import KMeans
import joblib as jbl
import numpy as np
import tensorflow as tf

tf.random.set_seed(42)
np.random.seed(42)
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

class RBFNet:
    
    """
    Implementation of Radial Basis Function Neural Networks (from input to hidden layer)
    """

    def __init__(self, X, y, X_test, y_test, n_hidden_neurons, width_factor) -> None:
        self.X = X
        self.y = y
        self.X_test = X_test
        self.y_test = y_test
        self.n_hidden = n_hidden_neurons
        self.p = width_factor
        
        self._centroids = None
        self._sigma_mat = None
        self._out_mat = None
        self._out_mat_test = None
        self._distance_mat = None
        
    def KMeans(self):
        km = KMeans(n_clusters=self.n_hidden).fit(self.X, self.y)
        self._centroids = km.cluster_centers_
        
    def gaussian_rbf(self):
        
        def get_min_indices(x, n=2):
            min_values = np.sort(x)[:n]
            return np.array([np.where(x == i)[0][0] for i in min_values])
        
        distance_mat = np.zeros((self.n_hidden, self.n_hidden))
        sigma_mat = np.zeros(self.n_hidden)
        
        # distance matrix calculation
        for i in range(self.n_hidden):
            for j in range(self.n_hidden):
                distance_mat[i][j] = np.linalg.norm(self._centroids[i] - self._centroids[j])
                distance_mat[j][i] = distance_mat[i][j]
        
        self._distance_mat = distance_mat
           
        # calculating sigma matrix
        for i in range(self.n_hidden):
            x = distance_mat[i]
            min_indices = get_min_indices(x, self.p)
            
            ED_array = np.zeros(self.p-1)
            
            for j in range(len(ED_array)):
                ED_array[j] = np.linalg.norm(self._centroids[i] - self._centroids[min_indices[j+1]])
            sigma_mat[i] = np.sqrt(np.square(ED_array).sum() / self.p)
            
        self._sigma_mat = sigma_mat
        self._out_mat = self.out_matrix(self.X, self.n_hidden, self._centroids, self._sigma_mat)
        self._out_mat_test = self.out_matrix(self.X_test, self.n_hidden, self._centroids, self._sigma_mat)
                
    def out_matrix(self, X, n_hidden, centroids, sigma_mat):
        out_mat = np.zeros((X.shape[0], n_hidden))
        for i in range(len(X)):
            for j in range(n_hidden):
                numerator = -np.square(X[i] - centroids[j]).sum()
                denominator = 2*(np.square(sigma_mat[j]))
                out_mat[i][j] = np.exp(numerator/denominator)
        return out_mat
    
    
class RBFNN(RBFNet):
    
    """
    Implementation of Radial Basis Function Neural Networks (hidden layer to output)
    """
    
    def __init__(self, X, y, X_test, y_test, n_hidden_neurons, width_factor, activation="linear"):
        super().__init__(X, y, X_test, y_test, n_hidden_neurons, width_factor)
        self.history = None
        
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(13, input_shape=(self.n_hidden, ), activation=activation)
        ])
        
    def summary(self):
        return self.model.summary()
        
    def fit(self, epochs=100, batch_size=32, optim=tf.optimizers.Adam, lr=0.1, loss_fn=tf.losses.MeanSquaredError, metrics=['mae']):
        
        self.KMeans()
        self.gaussian_rbf()
        self.model.compile(optim(learning_rate=lr), loss=loss_fn(), metrics=metrics)
        self.history = self.model.fit(x=self._out_mat, y=self.y, validation_data=(self._out_mat_test, self.y_test), epochs=epochs, batch_size=batch_size, callbacks=[WandbMetricsLogger()])
        
        self.model.save(f"RBF_ANN_model_bs_{batch_size}_N_{self.n_hidden}_P_{self.p}.h5")
        export_metadata = {
            "neurons" : self.n_hidden,
            "centers" : self._centroids,
            "sigma" : self._sigma_mat,
            "batch_size" : batch_size,
            "lr" : lr,
            "loss_fn" : loss_fn
        }

        jbl.dump(export_metadata, f"Metadata_N_{self.n_hidden}_P_{self.p}_bs_{batch_size}.jbl")
        
    def predict(self, X:np.ndarray, rbf_file:str=None, rbf_ann_file:str=None):
        if (rbf_ann_file == None or rbf_file == None):
            centers = self._centroids
            sigma = self._sigma_mat
            out_mat = self.out_matrix(X=X, n_hidden=self.n_hidden, centroids=centers, sigma_mat=sigma)[-1].reshape((1, self.n_hidden))
            return self.model.predict(out_mat)
        else:
            metadata = jbl.load(rbf_file)
            centers = metadata['centers']
            sigma = metadata['sigma']
            n_hidden = metadata['neurons']
            out_mat = self.out_matrix(X=X, n_hidden=n_hidden, centroids=centers, sigma_mat=sigma)[-1].reshape((1, n_hidden))
            return tf.keras.models.load_model(rbf_ann_file).predict(out_mat)

ModuleNotFoundError: ignored